In [47]:
# Importing all the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\manik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
reviews_df = pd.read_json(r"C:\Users\manik\Downloads\kindle_reviews\kindle_reviews.json", lines=True)
reviews_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1F6404F1VG29J,B000F83SZQ,Avidreader,"[0, 0]",I enjoy vintage books and movies so I enjoyed ...,5,Nice vintage story,1399248000,"05 5, 2014"
1,AN0N05A9LIJEQ,B000F83SZQ,critters,"[2, 2]",This book is a reissue of an old one; the auth...,4,Different...,1388966400,"01 6, 2014"
2,A795DMNCJILA6,B000F83SZQ,dot,"[2, 2]",This was a fairly interesting read. It had ol...,4,Oldie,1396569600,"04 4, 2014"
3,A1FV0SX13TWVXQ,B000F83SZQ,"Elaine H. Turley ""Montana Songbird""","[1, 1]",I'd never read any of the Amy Brewster mysteri...,5,I really liked it.,1392768000,"02 19, 2014"
4,A3SPTOKDG7WBLN,B000F83SZQ,Father Dowling Fan,"[0, 1]","If you like period pieces - clothing, lingo, y...",4,Period Mystery,1395187200,"03 19, 2014"


**Data Dictionary**<br>
asin - ID of the product <br>
helpful - helpfulness rating of the review <br>
overall - rating of the product <br>
reviewText - text of the review (heading) <br>
reviewTime - time of the review (raw) <br>
reviewerID - ID of the reviewer <br>
reviewerName - name of the reviewer <br>
summary - summary of the review<br>
unixReviewTime - unix timestamp <br>

In [49]:
unique_items_count = reviews_df['asin'].nunique()
print(unique_items_count)

61934


In [50]:
len(reviews_df)

982619

In [51]:
product_counts = reviews_df["asin"].value_counts()
products_filtered = product_counts[product_counts>=50]

In [52]:
reviews_df_filtered = reviews_df.query("asin in @products_filtered.index.tolist()")

In [53]:
reviews_df_filtered

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
189,A19V8I1LEHA1YW,B000JMLBHU,"Alexander Hollins ""Author, Reader, Editor, Tired""","[0, 0]",A manual on desert island survival in its own ...,4,A classic for a reason,1404000000,"06 29, 2014"
190,A3J5D9XN0QT9AG,B000JMLBHU,Allrie,"[0, 0]",I remember going to the movie theatre when abo...,5,Trouble,1367366400,"05 1, 2013"
191,A33EX8MZYBV0YI,B000JMLBHU,alone with books,"[0, 0]","this was an Ok book, but slow at times, I actu...",3,the Mysterrous Island,1374537600,"07 23, 2013"
192,A3HK1FZVP03IIC,B000JMLBHU,Amazon Customer,"[0, 0]",Classic. What else needs to be said. An advent...,4,Classic,1358208000,"01 15, 2013"
193,A27EEX3H2CB70G,B000JMLBHU,Amazon Customer,"[0, 0]",I read the who thing on my phone. Enjoyed it ...,4,Good Story,1365638400,"04 11, 2013"
...,...,...,...,...,...,...,...,...,...
982567,A20MYFNLY6TGNL,B00LZKMXBI,"The Book Fairy ""The Book Fairy""","[0, 0]",Amazing! I was crying from page 1 And sweating...,5,Tears galore!!!!,1406073600,"07 23, 2014"
982568,A19DWIC1T7127Y,B00LZKMXBI,T,"[3, 3]",I received an advanced reader's copy of this b...,4,Moving On,1405900800,"07 21, 2014"
982569,A28E8S7CY2LJ5W,B00LZKMXBI,Tlh,"[1, 1]","Grab tissues, the first few pages had me cryin...",5,Grab your tissues and enjoy a great story !!!!!,1405900800,"07 21, 2014"
982570,A2GCYN1TPTKX72,B00LZKMXBI,Tricia,"[4, 5]",ARC review..... Let me just start by saying I ...,5,Awesome!!!!!,1405900800,"07 21, 2014"


In the filtered dataframe we are keeping only products that are reviews atleast 50 times

The objective was to build a recommendation system based on the reviews. Now we want to process our dataset, our thoughts was to sample the data from the helpful column which says out of the people who saw the review how many found it helpful, this is our first thought. 

In [54]:
reviews_df["helpful"].iloc[0]

[0, 0]

now I want to check for the ones which are more helpful based on the percentage

In [55]:
reviews_df_filtered["help_percentage"] = reviews_df_filtered["helpful"].apply(lambda x : x[0]/x[1] if x[1]!=0 else 1)

C:\Users\manik\AppData\Local\Temp\ipykernel_31704\3949530801.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df_filtered["help_percentage"] = reviews_df_filtered["helpful"].apply(lambda x : x[0]/x[1] if x[1]!=0 else 1)


In [56]:
reviews_df_filtered = reviews_df_filtered.query("help_percentage>=0.8").reset_index(drop =True)

In [57]:
reviews_df_filtered.shape

(222231, 10)

In [58]:
# Now again we filtered those reviews which are really helpful

In [59]:
reviews_df_filtered.reset_index(drop=True, inplace = True)

In [60]:
reviews_df_filtered.isna().sum()

reviewerID            0
asin                  0
reviewerName       2016
helpful               0
reviewText            0
overall               0
summary               0
unixReviewTime        0
reviewTime            0
help_percentage       0
dtype: int64

In [61]:
# Unicode Normalization 
import unicodedata
normalized_text = reviews_df_filtered['reviewText'].apply(lambda x: unicodedata.normalize('NFC', x))
reviews_df_filtered['reviewText'] = normalized_text

In [62]:
reviews_df_filtered

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,help_percentage
0,A19V8I1LEHA1YW,B000JMLBHU,"Alexander Hollins ""Author, Reader, Editor, Tired""","[0, 0]",A manual on desert island survival in its own ...,4,A classic for a reason,1404000000,"06 29, 2014",1.000000
1,A3J5D9XN0QT9AG,B000JMLBHU,Allrie,"[0, 0]",I remember going to the movie theatre when abo...,5,Trouble,1367366400,"05 1, 2013",1.000000
2,A33EX8MZYBV0YI,B000JMLBHU,alone with books,"[0, 0]","this was an Ok book, but slow at times, I actu...",3,the Mysterrous Island,1374537600,"07 23, 2013",1.000000
3,A3HK1FZVP03IIC,B000JMLBHU,Amazon Customer,"[0, 0]",Classic. What else needs to be said. An advent...,4,Classic,1358208000,"01 15, 2013",1.000000
4,A27EEX3H2CB70G,B000JMLBHU,Amazon Customer,"[0, 0]",I read the who thing on my phone. Enjoyed it ...,4,Good Story,1365638400,"04 11, 2013",1.000000
...,...,...,...,...,...,...,...,...,...,...
222226,A163XZU3HM4EHV,B00LZKMXBI,Stacy Hahn,"[14, 17]",This book was absolutely perfect! Second Chanc...,5,I Loved it!!!,1405900800,"07 21, 2014",0.823529
222227,A20MYFNLY6TGNL,B00LZKMXBI,"The Book Fairy ""The Book Fairy""","[0, 0]",Amazing! I was crying from page 1 And sweating...,5,Tears galore!!!!,1406073600,"07 23, 2014",1.000000
222228,A19DWIC1T7127Y,B00LZKMXBI,T,"[3, 3]",I received an advanced reader's copy of this b...,4,Moving On,1405900800,"07 21, 2014",1.000000
222229,A28E8S7CY2LJ5W,B00LZKMXBI,Tlh,"[1, 1]","Grab tissues, the first few pages had me cryin...",5,Grab your tissues and enjoy a great story !!!!!,1405900800,"07 21, 2014",1.000000


In [63]:
# Remove unwanted spaces,extra trailing and leading spaces,punctuations,full-stops,inverted commas,special characters
import re
def clean_text(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text) 
    return cleaned_text.strip()
reviews_df_filtered['reviewText'] = reviews_df_filtered['reviewText'].apply(lambda x: clean_text(x))


In [64]:
# Removing stopwords from the dataset
def remove_stopwords(text):
    tokens = word_tokenize(text)    
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text
reviews_df_filtered['reviewText'] = reviews_df_filtered['reviewText'].apply(lambda x: remove_stopwords(x))


KeyboardInterrupt: 

In [ ]:
# Remove unwanted spaces,extra trailing and leading spaces,punctuations,full-stops,inverted commas,special characters
import re
def clean_text(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text) 
    return cleaned_text.strip()
reviews_df_filtered['reviewText'] = reviews_df_filtered['reviewText'].apply(lambda x: clean_text(x))


In [ ]:
reviews_df_filtered["reviewText"].iloc[0]

In [ ]:
filtered_product = reviews_df_filtered.query("asin=='B000JMLBHU'")

In [ ]:
filtered_product

In [ ]:
from transformers import pipeline

# Load sentiment analysis pipeline
classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased')


In [ ]:
filtered_product["truncated_text"] = filtered_product["reviewText"].apply(lambda x: classifier.tokenizer.tokenize(x))

In [ ]:
filtered_product

In [ ]:
filtered_product["truncated_text"] = filtered_product["truncated_text"].apply(lambda x: ' '.join(x[:400]))

In [ ]:
filtered_product["sentiment_output"] = filtered_product["truncated_text"].apply(lambda x: classifier(x))

In [ ]:
filtered_product

In [ ]:
def extract_label_score(sentiment_dict):
    
    if sentiment_dict and isinstance(sentiment_dict, list):
        return sentiment_dict[0]['label'], sentiment_dict[0]['score']

In [ ]:
filtered_product[['label', 'score']] = filtered_product['sentiment_output'].apply(lambda x: pd.Series(extract_label_score(x)))

In [ ]:
filtered_product

In [ ]:
filtered_product.query("label=='POSITIVE'")

In [ ]:
filtered_product

In [ ]:
filtered_product.iloc[1,4]

In [41]:
reviews_df.query("asin=='B000JMLBHU'").iloc[1,4]

"I remember going to the movie theatre when about 56/7 yrs old. with my cousin 4 years older.  We enjoyed it so much that we staye through a second showing--and got into huge trouble!  I saw the book's availability and realized that I'd never actually read the book.  It did not disappoint.  At our upcoming family reunion, I'd like to share that movie experience with my grandkids--but am sending them the book to enjoy first."